# Momentum Portfolios #

In [10]:
# Import Libraries

# Data Management
import pandas as pd

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from portfolios_toolkit import calculate_analytics
from capm_toolkit import capm_regression
from other_data_functions import rolling_calc_rstr

In [34]:
# Import Data
df_returns = pd.read_csv(r'..\additional_data\stocks_returns.csv')
df_returns = df_returns.rename(columns={'Unnamed: 0':'Date'})
df_returns.set_index('Date', inplace=True)
df_returns.index = pd.to_datetime(df_returns.index)
df_returns = df_returns.dropna(axis=1)

df_returns

In [35]:
# Import the Alphas
df_alphas = pd.read_csv(r'..\additional_data\capm_halpha.csv')
df_alphas.set_index('date', inplace=True)
df_alphas.index = pd.to_datetime(df_alphas.index)
df_alphas = df_alphas[df_returns.columns]

df_alphas

In [36]:
# Create the RS DFs
relative_strenght_long = rolling_calc_rstr(
    df_returns,
    window_size=252,
    half_life=126
).T

relative_strenght_short = rolling_calc_rstr(
    df_returns,
    window_size=28,
    half_life=14,
    min_obs=13
).T

In [37]:
# Calculate the Relative Strenght Index
relative_strenght = (relative_strenght_long - relative_strenght_short).dropna()
relative_strenght = relative_strenght.loc[df_alphas.index]

relative_strenght

In [38]:
# Now we are going to z-score them
def standardize_zscore(
        variable: pd.DataFrame
) -> pd.DataFrame:
    # Calculate Mean
    mean = variable.mean(axis=1)
    
    # Calculate Cross-Sectional Standard Deviation
    std = variable.std(axis=1)
    
    # Standardize (broadcasting Series across DataFrame rows)
    zscore_df = (variable.subtract(mean, axis = 0)).divide(std, axis = 0)
    
    return zscore_df

In [39]:
# Z-Score Alphas
alphas_zscore_df = standardize_zscore(df_alphas)

alphas_zscore_df

In [40]:
# Z-Score Relative Strenght
rstr_zscore_df = standardize_zscore(relative_strenght)

rstr_zscore_df

In [41]:
# Now Join both
momentum_df = (alphas_zscore_df + rstr_zscore_df)/2 

momentum_df

In [42]:
# And Z-Score it
momentum_zscore_df = standardize_zscore(momentum_df)

momentum_zscore_df

### Calculate the Weights ###

In [78]:
# Function to calculate the weights (as we saw above)
def calculate_momentum_weights(
    momentum: pd.Series
) -> pd.Series:

    # Ranking Momentum
    ranks = momentum.rank()
    z_bar = ranks.mean()
    median_rank = ranks.median()
    
    # Split using the Median
    low_mom = ranks[ranks < median_rank]
    high_mom = ranks[ranks >= median_rank]

    # Calculate Weights
    w_low = (z_bar - low_mom).clip(lower=0)
    w_high = (high_mom - z_bar).clip(lower=0)
    
    # Standardize Weights
    w_low /= w_low.sum()
    w_high /= w_high.sum()
    
    # Concat
    mom_weights = pd.concat([-w_low, w_high])

    return mom_weights

In [79]:
# Let's make a try:
momentum_weights = calculate_momentum_weights(momentum_df.iloc[0])

In [100]:
# Rolling weights
# Function for the Rolling Weights
def calculate_mom_rolling_weights(
    mom_df, 
    rebalance_days=21
):
    # Rebalancing Dates
    rebalance_dates = mom_df.index[::rebalance_days]

    # List used for storing
    weights_list = []

    # Loop
    for date in rebalance_dates:
        # Betas for each date
        momentum_today = mom_df.loc[date]
        
        # Calculate Weights and store them
        weights = calculate_momentum_weights(momentum_today)
        weights.name = date
        weights_list.append(weights)

    # Create a DataFrame
    mom_weights_rebalance = pd.DataFrame(weights_list)

    # Reindexing for daily weights
    mom_weights_daily = mom_weights_rebalance.reindex(mom_df.index)

    # Forward Fill
    mom_weights_daily = mom_weights_daily.ffill().fillna(0)

    # Reindexing Columns to have consistency
    mom_weights_daily = mom_weights_daily.reindex(columns=mom_df.columns).fillna(0)

    return mom_weights_daily

In [112]:
# Get the Weights
mom_daily_weights = calculate_mom_rolling_weights(momentum_zscore_df, rebalance_days=63)

mom_daily_weights

In [113]:
# Calculate the Portfolio using rebalancing
mom_rebalancing_portfolio = (df_returns.loc[mom_daily_weights.index] * mom_daily_weights).sum(axis = 1)
mom_rebalancing_portfolio.name = 'mom_rebalancing_portfolio'

mom_rebalancing_portfolio

In [114]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(mom_rebalancing_portfolio.cumsum().mul(100), label='MOM Strategy with Rebalacing', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

In [115]:
# We can download the premiums in the Fama and French website
momemtum_prime = pd.read_csv(r'..\additional_data\famafrench_momentum.csv')
momemtum_prime.set_index('Date', inplace=True)
momemtum_prime.index = pd.to_datetime(momemtum_prime.index)
momemtum_prime = momemtum_prime.div(100)
momemtum_prime.columns = ['momentum']

momemtum_prime

In [116]:
# Create Plot
plt.figure(figsize=(10, 6))
plt.plot(mom_rebalancing_portfolio.cumsum().mul(100), label='MOM Strategy with Rebalacing', alpha=1)
plt.plot(momemtum_prime.loc['2016':'2024'].cumsum().mul(100), label='MOM Fama and French Premium', alpha=1)

# Config
plt.title('Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid()

# Show
plt.show() 

In [117]:
# Calculate the Portfolio Correlation
mom_rebalancing_portfolio.corr(momemtum_prime['momentum'])

Why is our portfolio so close to the Fama & French momentum premium? What they calculate is the Winners-minus-Losers premium. In our portfolio, we are shorting the loser stocks and betting in favor of the winner stocks. So, naturally, the results are very similar.